<a href="https://colab.research.google.com/github/AbhayMudgal/Dogs-vs-Cats/blob/main/Cats_vs_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


In [38]:
import io, os
import numpy as np
import pandas as pd
import json
import random
random.seed(1)

In [3]:
kaggle_json = {"username":"abhay1010","key":"4af3df22394b3bfc76ea750845f8b173"}
os.makedirs("/content/.kaggle/", exist_ok = True)
os.makedirs("/root/.kaggle/", exist_ok = True)
with open('/content/.kaggle/kaggle.json', 'w') as file:
  json.dump(kaggle_json, file)
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json

In [4]:
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
%ls /content/.kaggle/

kaggle.json


In [6]:
! kaggle competitions download -c dogs-vs-cats

 99% 540M/543M [00:04<00:00, 152MB/s]
100% 543M/543M [00:04<00:00, 124MB/s]
100% 270M/271M [00:04<00:00, 50.8MB/s]
100% 271M/271M [00:04<00:00, 58.8MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 88.4MB/s]


In [ ]:
! unzip /content/train.zip 

In [ ]:
! unzip /content/test1.zip

In [9]:
import random
import tensorflow as tf
from tensorflow import keras as K
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [10]:
# Parameters
val_split = 0.2
size = 150
Batch_size = 32

In [11]:
main = 'data/'
subdir = ['train/', 'test/']
for sub in subdir:
  label_dir = ['dogs/', 'cats/']
  for label in label_dir:
    new_dir = main + sub + label
    os.makedirs(new_dir, exist_ok = True)

src_dir = '/content/train'
for file in os.listdir(src_dir):
  src = src_dir + '/' + file
  tar_dir = 'train/'
  if random.random()<val_split:
    tar_dir = 'test/'
  word = file.split('.')[0]
  if word == 'cat':
    target = main + tar_dir + 'cats/' + file
    copyfile(src, target)
  elif word == 'dog':
    target = main + tar_dir + 'dogs/' + file
    copyfile(src, target)  


In [15]:
files = os.listdir('data/train/dogs')
print(len(files))

10058


In [17]:
train_generator = ImageDataGenerator(rescale = 1/255.0, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')
test_generator = ImageDataGenerator(rescale = 1/255.0)

In [20]:
train_gen = train_generator.flow_from_directory('data/train', class_mode = 'binary', batch_size = Batch_size, target_size = (size, size))
val_gen = test_generator.flow_from_directory('data/test', class_mode = 'binary', batch_size = Batch_size, target_size = (size, size))

Found 19968 images belonging to 2 classes.
Found 5032 images belonging to 2 classes.


In [21]:
print(train_gen.class_indices, val_gen.class_indices)

{'cats': 0, 'dogs': 1} {'cats': 0, 'dogs': 1}


In [28]:
CNN = K.models.Sequential([
  Conv2D(32, (3, 3), activation = 'relu', input_shape = (size, size, 3)),
  MaxPool2D(2, 2),
  Dropout(0.2),
  Conv2D(64, (3, 3), activation = 'relu'),
  MaxPool2D(2, 2),
  Dropout(0.2),
  Conv2D(128, (3, 3), activation = 'relu'),
  MaxPool2D(2, 2),
  Dropout(0.2),
  Flatten(),
  Dense(512, activation = 'relu'),
  Dropout(0.2),
  Dense(64, activation = 'relu'),
  Dense(1, activation = 'sigmoid')                       
])
                    

In [29]:
opt = K.optimizers.Adam(learning_rate=0.001)
CNN.compile(optimizer=opt, loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
history = CNN.fit(train_gen, epochs = 20, validation_data=val_gen)

In [ ]:
from keras.applications.vgg16 import VGG16

pre_trained = VGG16(include_top = False, weights = 'imagenet', input_shape = (size, size, 3))

for layer in pre_trained.layers:
  layer.trainable = False

x = Flatten()(pre_trained.layers[-1].output)
x = Dense(256, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(1, activation = 'sigmoid')(x)

VGG = K.Model(pre_trained.inputs, x)

VGG.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

run_vgg = VGG.fit(train_gen, epochs = 10, validation_data = val_gen)

In [48]:
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt

predictions = []
count = 1
  # predicting images
for fn in os.listdir('/content/test1/'):
  path = '/content/test1/' + fn
  img = image.load_img(path, target_size=(size, size))
  x = image.img_to_array(img)
  
  x = np.expand_dims(x, axis=0)
  pred_img = VGG.predict(x)
  
  if pred_img == 1:
    y_pred = 1
  else:
    y_pred = 0 
  predictions.append(y_pred)



TypeError: ignored

In [49]:
df = pd.DataFrame(predictions)
df.to_csv('outputs.csv')

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/mew.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'